# Testing All Models on Adjusted Dataset

In [1]:
import os
import csv
import json
import math
import torch
import argparse
import difflib
import logging
import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertForMaskedLM
from transformers import AlbertTokenizer, AlbertForMaskedLM
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
from transformers import AutoModel, AutoTokenizer
# 
from collections import defaultdict
from tqdm import tqdm

from crows_pairs_methods import *

In [2]:
BERT_models = [
    'bert-base-cased',
    'bert-base-uncased',
    'bert-large-uncased',
    'bert-large-cased',
    'bert-base-multilingual-uncased',
    'bert-base-multilingual-cased',
    'allenai/scibert_scivocab_uncased',
    'emilyalsentzer/Bio_ClinicalBERT',
    'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract',
    'ProsusAI/finbert',
    'nlpaueb/legal-bert-base-uncased',
    'GroNLP/hateBERT',
    'anferico/bert-for-patents',
    'jackaduma/SecBERT'
]

ALBERT_models = [
    'albert-base-v1',
    'albert-base-v2'
]

ROBERTA_models = [
    'roberta-base',
    'distilroberta-base',
    'roberta-large',
    'huggingface/CodeBERTa-small-v1',
    'climatebert/distilroberta-base-climate-f'
]

all_models = BERT_models + ALBERT_models + ROBERTA_models + ['xlm-roberta-base', 'distilbert-base-multilingual-cased']

In [3]:
bias_types = [
    'Race',
    'Gender',
    'Socio-Economic', 
    'Nationality', 
    'Religion', 
    'Age', 
    'Sexual Orientation', 
    'Physical Appearance', 
    'Disability'
]

In [4]:
bias_types = [
    'race-color',
    'gender',
    'socioeconomic',
    'nationality',
    'religion', 
    'age',
    'sexual-orientation',
    'physical-appearance',
    'disability'
]

In [5]:
empty_dataframe_dictionary = {
    'models' : [],
    'metric_scores' : [],
    'stereotype_scores' : [],
    'antistereotype_scores' : []
}

dataframe_dictionary_race = empty_dataframe_dictionary
dataframe_dictionary_gender = empty_dataframe_dictionary
dataframe_dictionary_socioeconomic = empty_dataframe_dictionary
dataframe_dictionary_nationality = empty_dataframe_dictionary
dataframe_dictionary_religion = empty_dataframe_dictionary
dataframe_dictionary_age = empty_dataframe_dictionary
dataframe_dictionary_sexualorientation = empty_dataframe_dictionary
dataframe_dictionary_physicalappearance = empty_dataframe_dictionary
dataframe_dictionary_disability = empty_dataframe_dictionary

In [11]:
dataframe_dictionary_race['models'] = dataframe_dictionary_race['models'][1:]

In [12]:
pd.DataFrame(dataframe_dictionary_race)

,models,metric_scores,stereotype_scores,antistereotype_scores
0,bert-base-cased,55.56,50.0,100.0
1,bert-base-uncased,55.56,62.5,0.0
2,bert-large-uncased,55.56,50.0,100.0
3,bert-large-cased,55.56,62.5,0.0
4,bert-base-multilingual-uncased,44.44,50.0,0.0
5,bert-base-multilingual-cased,77.78,75.0,100.0
6,allenai/scibert_scivocab_uncased,33.33,37.5,0.0
7,emilyalsentzer/Bio_ClinicalBERT,44.44,37.5,100.0
8,microsoft/BiomedNLP-PubMedBERT-base-uncased-ab...,55.56,62.5,0.0
9,ProsusAI/finbert,66.67,75.0,0.0


In [8]:
logging.basicConfig(level=logging.INFO)

for bias_type in bias_types:
    
    # load data into panda DataFrame
    df_data = read_data("fixed_data.csv")

    # Filtering to Disability Data
    df_data = df_data[df_data['bias_type']==bias_type][:9]

    for model_name in all_models:

        # supported masked language models (using bert)
        if model_name in BERT_models:
            tokenizer = BertTokenizer.from_pretrained(model_name)
            model = BertForMaskedLM.from_pretrained(model_name)
        elif model_name in ALBERT_models:
            tokenizer = AlbertTokenizer.from_pretrained(model_name)
            model = AlbertForMaskedLM.from_pretrained(model_name)
        elif model_name in ROBERTA_models:
            tokenizer = RobertaTokenizer.from_pretrained(model_name)
            model = RobertaForMaskedLM.from_pretrained(model_name)
        elif model_name == 'xlm-roberta-base':
            tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
            model = XLMRobertaForMaskedLM.from_pretrained(model_name)
        elif model_name == 'distilbert-base-multilingual-cased':
            tokenizer = DistilBertTokenizer.from_pretrained(model_name)
            model = DistilBertForMaskedLM.from_pretrained(model_name)

        mask_token = tokenizer.mask_token
        log_softmax = torch.nn.LogSoftmax(dim=0)
        vocab = tokenizer.get_vocab()
        with open("bert" + ".vocab", "w") as f:
            f.write(json.dumps(vocab))

        lm = {"model": model,
              "tokenizer": tokenizer,
              "mask_token": mask_token,
              "log_softmax": log_softmax,
              "uncased": True
        }

        # score each sentence. 
        # each row in the dataframe has the sentid and score for pro and anti stereo.
        df_score = pd.DataFrame(columns=['sent_more', 'sent_less', 
                                         'sent_more_score', 'sent_less_score',
                                         'score', 'stereo_antistereo', 'bias_type'], dtype=object)

        total_stereo, total_antistereo = 0, 0
        stereo_score, antistereo_score = 0, 0

        N = 0
        neutral = 0
        total = len(df_data.index)
        with tqdm(total=total) as pbar:
            for index, data in df_data.iterrows():
                direction = data['direction']
                bias = data['bias_type']
                score = mask_unigram(data, lm)

                for stype in score.keys():
                    score[stype] = round(score[stype], 3)

                N += 1
                pair_score = 0
                pbar.update(1)
                if score['sent1_score'] == score['sent2_score']:
                    neutral += 1
                else:
                    if direction == 'stereo':
                        total_stereo += 1
                        if score['sent1_score'] > score['sent2_score']:
                            stereo_score += 1
                            pair_score = 1
                    elif direction == 'antistereo':
                        total_antistereo += 1
                        if score['sent2_score'] > score['sent1_score']:
                            antistereo_score += 1
                            pair_score = 1

                sent_more, sent_less = '', ''
                if direction == 'stereo':
                    sent_more = data['sent1']
                    sent_less = data['sent2']
                    sent_more_score = score['sent1_score']
                    sent_less_score = score['sent2_score']
                else:
                    sent_more = data['sent2']
                    sent_less = data['sent1']
                    sent_more_score = score['sent2_score']
                    sent_less_score = score['sent1_score']

                df_score = df_score.append({'sent_more': sent_more,
                                            'sent_less': sent_less,
                                            'sent_more_score': sent_more_score,
                                            'sent_less_score': sent_less_score,
                                            'score': pair_score,
                                            'stereo_antistereo': direction,
                                            'bias_type': bias
                                          }, ignore_index=True)

        if bias_type == 'race-color':
            dataframe_dictionary_race['models'].append(model_name)
            dataframe_dictionary_race['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_race['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_race['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'gender':
            dataframe_dictionary_gender['models'].append(model_name)
            dataframe_dictionary_gender['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_gender['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_gender['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'socioeconomic':
            dataframe_dictionary_socioeconomic['models'].append(model_name)
            dataframe_dictionary_socioeconomic['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_socioeconomic['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_socioeconomic['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'nationality':
            dataframe_dictionary_nationality['models'].append(model_name)
            dataframe_dictionary_nationality['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_nationality['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_nationality['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'religion':
            dataframe_dictionary_religion['models'].append(model_name)
            dataframe_dictionary_religion['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_religion['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_religion['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'age':
            dataframe_dictionary_age['models'].append(model_name)
            dataframe_dictionary_age['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_age['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_age['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'sexual-orientation':
            dataframe_dictionary_sexualorientation['models'].append(model_name)
            dataframe_dictionary_sexualorientation['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_sexualorientation['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_sexualorientation['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'physical-appearance':
            dataframe_dictionary_physicalappearance['models'].append(model_name)
            dataframe_dictionary_physicalappearance['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_physicalappearance ['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_physicalappearance  ['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))          
        
        elif bias_type == 'disability':
            dataframe_dictionary_disability['models'].append(model_name)
            dataframe_dictionary_disability['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_disability['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_disability['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/369k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/321M [00:00<?, ?B/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:17<00:00,  1.90s/it]


ImportError: 
AlbertTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment.


In [14]:
for dataframe_dict in [dataframe_dictionary_race,
                       dataframe_dictionary_gender,
                       dataframe_dictionary_socioeconomic,
                       dataframe_dictionary_nationality,
                       dataframe_dictionary_religion,
                       dataframe_dictionary_age,
                       dataframe_dictionary_sexualorientation,
                       dataframe_dictionary_physicalappearance,
                       dataframe_dictionary_disability]:
    print(pd.DataFrame(dataframe_dict))

                                               models  metric_scores  \
0                                     bert-base-cased          55.56   
1                                   bert-base-uncased          55.56   
2                                  bert-large-uncased          55.56   
3                                    bert-large-cased          55.56   
4                      bert-base-multilingual-uncased          44.44   
5                        bert-base-multilingual-cased          77.78   
6                    allenai/scibert_scivocab_uncased          33.33   
7                     emilyalsentzer/Bio_ClinicalBERT          44.44   
8   microsoft/BiomedNLP-PubMedBERT-base-uncased-ab...          55.56   
9                                    ProsusAI/finbert          66.67   
10                    nlpaueb/legal-bert-base-uncased          77.78   
11                                    GroNLP/hateBERT          55.56   
12                          anferico/bert-for-patents          5

In [13]:
% pip install sentencepiece

UsageError: Line magic function `%` not found.


In [23]:
logging.basicConfig(level=logging.INFO)

for bias_type in bias_types:
    
    # load data into panda DataFrame
    df_data = read_data("fixed_data.csv")

    # Filtering to Disability Data
    df_data = df_data[df_data['bias_type']==bias_type][:9]

    for model_name in ['albert-base-v1',
                       'albert-base-v2',
                       'roberta-base',
                       'distilroberta-base',
                       'roberta-large',
                       'huggingface/CodeBERTa-small-v1',
                       'climatebert/distilroberta-base-climate-f']:

        # supported masked language models (using bert)
        if model_name in BERT_models:
            tokenizer = BertTokenizer.from_pretrained(model_name)
            model = BertForMaskedLM.from_pretrained(model_name)
        elif model_name in ALBERT_models:
            tokenizer = AlbertTokenizer.from_pretrained(model_name)
            model = AlbertForMaskedLM.from_pretrained(model_name)
        elif model_name in ROBERTA_models:
            tokenizer = RobertaTokenizer.from_pretrained(model_name)
            model = RobertaForMaskedLM.from_pretrained(model_name)
        elif model_name == 'xlm-roberta-base':
            tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
            model = XLMRobertaForMaskedLM.from_pretrained(model_name)
        elif model_name == 'distilbert-base-multilingual-cased':
            tokenizer = DistilBertTokenizer.from_pretrained(model_name)
            model = DistilBertForMaskedLM.from_pretrained(model_name)

        mask_token = tokenizer.mask_token
        log_softmax = torch.nn.LogSoftmax(dim=0)
        vocab = tokenizer.get_vocab()
        with open("bert" + ".vocab", "w") as f:
            f.write(json.dumps(vocab))

        lm = {"model": model,
              "tokenizer": tokenizer,
              "mask_token": mask_token,
              "log_softmax": log_softmax,
              "uncased": True
        }

        # score each sentence. 
        # each row in the dataframe has the sentid and score for pro and anti stereo.
        df_score = pd.DataFrame(columns=['sent_more', 'sent_less', 
                                         'sent_more_score', 'sent_less_score',
                                         'score', 'stereo_antistereo', 'bias_type'], dtype=object)

        total_stereo, total_antistereo = 0, 0
        stereo_score, antistereo_score = 0, 0

        N = 0
        neutral = 0
        total = len(df_data.index)
        with tqdm(total=total) as pbar:
            for index, data in df_data.iterrows():
                direction = data['direction']
                bias = data['bias_type']
                score = mask_unigram(data, lm)

                for stype in score.keys():
                    score[stype] = round(score[stype], 3)

                N += 1
                pair_score = 0
                pbar.update(1)
                if score['sent1_score'] == score['sent2_score']:
                    neutral += 1
                else:
                    if direction == 'stereo':
                        total_stereo += 1
                        if score['sent1_score'] > score['sent2_score']:
                            stereo_score += 1
                            pair_score = 1
                    elif direction == 'antistereo':
                        total_antistereo += 1
                        if score['sent2_score'] > score['sent1_score']:
                            antistereo_score += 1
                            pair_score = 1

                sent_more, sent_less = '', ''
                if direction == 'stereo':
                    sent_more = data['sent1']
                    sent_less = data['sent2']
                    sent_more_score = score['sent1_score']
                    sent_less_score = score['sent2_score']
                else:
                    sent_more = data['sent2']
                    sent_less = data['sent1']
                    sent_more_score = score['sent2_score']
                    sent_less_score = score['sent1_score']

                df_score = df_score.append({'sent_more': sent_more,
                                            'sent_less': sent_less,
                                            'sent_more_score': sent_more_score,
                                            'sent_less_score': sent_less_score,
                                            'score': pair_score,
                                            'stereo_antistereo': direction,
                                            'bias_type': bias
                                          }, ignore_index=True)

        if bias_type == 'race-color':
            dataframe_dictionary_race['models'].append(model_name)
            dataframe_dictionary_race['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_race['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_race['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'gender':
            dataframe_dictionary_gender['models'].append(model_name)
            dataframe_dictionary_gender['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_gender['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_gender['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'socioeconomic':
            dataframe_dictionary_socioeconomic['models'].append(model_name)
            dataframe_dictionary_socioeconomic['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_socioeconomic['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_socioeconomic['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'nationality':
            dataframe_dictionary_nationality['models'].append(model_name)
            dataframe_dictionary_nationality['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_nationality['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_nationality['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'religion':
            dataframe_dictionary_religion['models'].append(model_name)
            dataframe_dictionary_religion['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_religion['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_religion['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'age':
            dataframe_dictionary_age['models'].append(model_name)
            dataframe_dictionary_age['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_age['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_age['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'sexual-orientation':
            dataframe_dictionary_sexualorientation['models'].append(model_name)
            dataframe_dictionary_sexualorientation['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_sexualorientation['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_sexualorientation['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))
        
        elif bias_type == 'physical-appearance':
            dataframe_dictionary_physicalappearance['models'].append(model_name)
            dataframe_dictionary_physicalappearance['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_physicalappearance ['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_physicalappearance  ['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))          
        
        elif bias_type == 'disability':
            dataframe_dictionary_disability['models'].append(model_name)
            dataframe_dictionary_disability['metric_scores'].append(round((stereo_score + antistereo_score) / N * 100, 2))
            dataframe_dictionary_disability['stereotype_scores'].append(round(stereo_score  / total_stereo * 100, 2))
            dataframe_dictionary_disability['antistereotype_scores'].append(round(antistereo_score  / total_antistereo * 100, 2))

TypeError: 'NoneType' object is not callable

In [17]:
model_name

'albert-base-v1'

In [25]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

TypeError: 'NoneType' object is not callable

In [27]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [21]:
import sentencepiece